In [12]:
import duckdb
con = duckdb.connect()

In [13]:
con.sql("LOAD postgres;")

In [14]:
con.sql("ATTACH 'host=localhost port=5432 dbname=postgres user=postgres password=postgres' AS postgres (TYPE postgres);")

### Populando Dim_Product

In [6]:
con.sql("""
    INSERT into postgres.gold.Dim_Product   
    select 
    	distinct 
    	hp.Product_ID
    	,hp.Dt_Load as Dt_Load_Hub
    	,hp.Source 
    	,sp.Product_Name 
    	,sp.Category_ID 
    	,sp.Category_Name 
    	,sp.Supplier_ID 
    	,sp.Supplier_Name
    	,sp.Dt_Load as Dt_Load_Sat
    from postgres.silver_raw_vault.HUB_Products hp
    inner join postgres.silver_raw_vault.SAT_Product sp on hp.Hash_Key_Product = sp.Hash_Key_Product
""")

In [7]:
con.sql("select * from postgres.gold.Dim_Product")

┌────────────┬─────────────────────┬─────────────────┬───┬─────────────┬──────────────────────┬─────────────────────┐
│ Product_ID │     Dt_Load_Hub     │     Source      │ … │ Supplier_ID │    Supplier_Name     │     Dt_Load_Sat     │
│   int32    │      timestamp      │     varchar     │   │   varchar   │       varchar        │      timestamp      │
├────────────┼─────────────────────┼─────────────────┼───┼─────────────┼──────────────────────┼─────────────────────┤
│          1 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ 1           │ Exotic Liquids       │ 2024-04-23 00:00:00 │
│          3 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ 1           │ Exotic Liquids       │ 2024-04-23 00:00:00 │
│          7 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ 3           │ Grandma Kelly's Ho…  │ 2024-04-23 00:00:00 │
│         25 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ 11          │ Heli Swaren GmbH &…  │ 2024-04-23 00:00:00 │
│         32 │ 2024-04-23 00:00:00 │ kaggle-postgres │ …

### Populando Dim_Employee

In [15]:
con.sql("""
    INSERT into postgres.gold.Dim_Employee   
    select 
    	distinct 
    	 he.Employee_ID
    	,he.Dt_Load as Dt_Load_Hub
    	,he.Source
    	,se.First_Name ||' '|| se.Last_Name as Name
    	,se.Title_Of_Courtesy 
    	,se.Birth_Date 
    	,se.Hire_Date 
    	,sea.Address 
    	,sea.City 
    	,sea.Region 
        ,se.Dt_Load as Dt_Load_Sat_Name
        ,sea.Dt_Load as Dt_Load_Sat_Address
    from postgres.silver_raw_vault.HUB_Employee he
    inner join postgres.silver_raw_vault.SAT_Employee se on he.Hash_Key_Employee = se.Hash_Key_Employee 
    inner join postgres.silver_raw_vault.SAT_Employee_Address sea on he.Hash_Key_Employee = sea.Hash_Key_Employee
""")

In [16]:
con.sql("select * from postgres.gold.Dim_Employee")

┌─────────────┬─────────────────────┬─────────────────┬───┬─────────┬─────────────────────┬─────────────────────┐
│ Employee_ID │     Dt_Load_Hub     │     Source      │ … │ Region  │  Dt_Load_Sat_Name   │ Dt_Load_Sat_Address │
│    int32    │      timestamp      │     varchar     │   │ varchar │      timestamp      │      timestamp      │
├─────────────┼─────────────────────┼─────────────────┼───┼─────────┼─────────────────────┼─────────────────────┤
│           9 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ NULL    │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│           4 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ WA      │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│           3 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ WA      │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│           8 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ WA      │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│           1 │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ WA      │ 2024-04-23 00:00:0

### Populando Dim_Customers

In [19]:
con.sql("""
    INSERT into postgres.gold.Dim_Customers  
    select distinct 
    	 hc.Customer_ID
    	 ,hc.Dt_Load as Dt_Load_Hub
    	 ,hc.Source 
    	 ,sc.Customer_Name 
    	 ,sc.Contact_Name 
    	 ,sca.Address 
    	 ,sca.City 
    	 ,sca.Region 
         ,sc.Dt_Load as Dt_Load_Sat_Name
    	 ,sca.Dt_Load as Dt_Load_Sat_Address
    from postgres.silver_raw_vault.HUB_Customers hc
    inner join postgres.silver_raw_vault.SAT_Customer sc on hc.Hash_Key_Customer = sc.Hash_Key_Customer 
    inner join postgres.silver_raw_vault.SAT_Customer_Address sca on hc.Hash_Key_Customer = sca.Hash_Key_Customer 

""")

In [20]:
con.sql("select * from postgres.gold.Dim_Customers")

┌─────────────┬─────────────────────┬─────────────────┬───┬───────────────┬─────────────────────┬─────────────────────┐
│ Customer_ID │     Dt_Load_Hub     │     Source      │ … │    Region     │  Dt_Load_Sat_Name   │ Dt_Load_Sat_Address │
│   varchar   │      timestamp      │     varchar     │   │    varchar    │      timestamp      │      timestamp      │
├─────────────┼─────────────────────┼─────────────────┼───┼───────────────┼─────────────────────┼─────────────────────┤
│ ALFKI       │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ NULL          │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ BOTTM       │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ BC            │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ CENTC       │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ NULL          │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ FAMIA       │ 2024-04-23 00:00:00 │ kaggle-postgres │ … │ SP            │ 2024-04-23 00:00:00 │ 2024-04-23 00:00:00 │
│ FOLKO       │ 2024-04-23 00:00:00 │ ka

### Populando Dim_Shipper

In [22]:
con.sql("""
    INSERT into postgres.gold.Dim_Shippers  
    select 
    	distinct
    	hs.Shipper_ID 
    	,hs.Dt_Load as Dt_Load_Hub
    	,hs.Source
    	,ss.Company_Name 
    	,ss.Phone 
    	,ss.Dt_Load as Dt_Load_Sat
    from postgres.silver_raw_vault.HUB_Shippers hs
    inner join postgres.silver_raw_vault.SAT_Shipper ss on hs.Hash_Key_Shipper = ss.Hash_Key_Shipper
""")

In [23]:
con.sql("select * from postgres.gold.Dim_Shippers")

┌────────────┬─────────────────────┬─────────────────┬──────────────────┬────────────────┬─────────────────────┐
│ Shipper_ID │     Dt_Load_Hub     │     Source      │   Company_Name   │     Phone      │     Dt_Load_Sat     │
│   int32    │      timestamp      │     varchar     │     varchar      │    varchar     │      timestamp      │
├────────────┼─────────────────────┼─────────────────┼──────────────────┼────────────────┼─────────────────────┤
│          3 │ 2024-04-23 00:00:00 │ kaggle-postgres │ Federal Shipping │ (503) 555-9931 │ 2024-04-24 00:00:00 │
│          1 │ 2024-04-23 00:00:00 │ kaggle-postgres │ Speedy Express   │ (503) 555-9831 │ 2024-04-24 00:00:00 │
│          2 │ 2024-04-23 00:00:00 │ kaggle-postgres │ United Package   │ (503) 555-3199 │ 2024-04-24 00:00:00 │
└────────────┴─────────────────────┴─────────────────┴──────────────────┴────────────────┴─────────────────────┘

### Populando Fato_Orders

In [33]:
con.sql("""
    INSERT into postgres.gold.Fact_Orders  
    select 
    	 b.Order_ID 
         ,strftime(so.Order_Date, '%Y%m%d') Date_ID
    	,b.Employee_ID 
    	,b.Customer_ID
    	,b.Product_ID
    	,b.Shipper_ID 
    	,so.Order_Date 
    	,so.required_date 
    	,so.shipped_date 
    	,sod.Quantity 
    	,sod.Unity_Price 
    	,sod.Discount 
    from postgres.silver_business_vault.Bridge_Store b
    inner join postgres.silver_raw_vault.SAT_Orders_Details sod on b.Hash_Key_Link_Products_Orders_Detail = sod.Hash_Key_Order_P_O 
    inner join postgres.silver_raw_vault.SAT_Orders so on b.Hash_Key_Orders = so.Hash_Key_Order 
    inner join postgres.silver_raw_vault.SAT_Product sp on b.Hash_Key_Product = sp.Hash_Key_Product 
    inner join postgres.silver_raw_vault.SAT_Customer sc on b.Hash_Key_Customer = sc.Hash_Key_Customer 
    inner join postgres.silver_raw_vault.SAT_Customer_Address sca on b.Hash_Key_Customer = sca.Hash_Key_Customer 
    inner join postgres.silver_raw_vault.SAT_Employee se on b.Hash_Key_Employee = se.Hash_Key_Employee 
    inner join postgres.silver_raw_vault.SAT_Employee_Address sea on b.Hash_Key_Employee = sea.Hash_Key_Employee 
    inner join postgres.silver_raw_vault.SAT_Shipper ss on b.Hash_Key_Shipper = ss.Hash_Key_Shipper 
""")

In [34]:
con.sql("select * from postgres.gold.Fact_Orders")

┌──────────┬──────────┬─────────────┬─────────────┬───┬─────────────────────┬──────────┬─────────────┬──────────┐
│ Order_ID │ Date_ID  │ Employee_ID │ Customer_ID │ … │    shipped_date     │ Quantity │ Unity_Price │ Discount │
│ varchar  │  int32   │    int32    │   varchar   │   │      timestamp      │  int32   │   double    │  double  │
├──────────┼──────────┼─────────────┼─────────────┼───┼─────────────────────┼──────────┼─────────────┼──────────┤
│ 10248    │ 20200704 │           5 │ VINET       │ … │ 2020-07-16 00:00:00 │       12 │        14.0 │      0.0 │
│ 10248    │ 20200704 │           5 │ VINET       │ … │ 2020-07-16 00:00:00 │       10 │         9.8 │      0.0 │
│ 10248    │ 20200704 │           5 │ VINET       │ … │ 2020-07-16 00:00:00 │        5 │        34.8 │      0.0 │
│ 10249    │ 20200705 │           6 │ TOMSP       │ … │ 2020-07-10 00:00:00 │       40 │        42.4 │      0.0 │
│ 10253    │ 20200710 │           3 │ HANAR       │ … │ 2020-07-16 00:00:00 │       20 │

In [35]:
con.close()